# Running this notebook

`bazel build lib/Transforms/s3:jupyter && ./bazel-bin/lib/Transforms/s3/jupyter`

# $S^3$, the 4-dimensional sphere
# Unit quaternions
Unit quaternions are also called rotation quaternions as they represent the 3D rotation group SO3.

# Rotate a vector using a point on the sphere
According to Euler's rotation theorem, any roations or sequence of rotations about a fixed point is equivalent to a single rotation by a given angle $\theta$ about a fixed axis. This axis is usually represented by a unit vector.

Quaternions are a way of storing this axis-angle representation in four numbers.

Then a rotation of angle $\theta$ around the axis defined by the unit vector $\vec{u}$ can be represented by the quaternion $q = cos(\frac{\theta}{2}) + (u_x i + u_y j + u_z k) sin(\frac{\theta}{2})$.

If we represent a 3D vector $p$ by a quaternion such that $p = p_x i + p_y j + p_z k$, then we can rotate it by $q$ by using the Hamilton product $p' = q p q^{-1}$.

In [1]:
# create a vector
# create a rotation quaternion
# rotate
from lib.Transforms.s3.s3 import S3
from lib.Math.Quaternions.quaternions import Quaternion
myvector = Quaternion([0, 1, 2, 3])
rot_quat = S3([0,1,0.5,0.1])

rotated_vector = rot_quat @ myvector @ rot_quat.inverse()
rotated_vector, myvector
# Step 2: implement the conversions quaternion - matrix

(q = -2.3592239273284576e-16 + 2.650793650793651i + -0.17460317460317465j + -2.6349206349206358k,
 q = 0 + 1i + 2j + 3k)

# Log map

In [1]:
from lib.Transforms.conversions.conversions import SO3_from_S3
# Go to tangent space from both a unit quaternion and a matrix
rot_mat = SO3_from_S3(rot_quat)
rot_mat.log_map()

/home/jules/documents/code/ComputerVision/bazel-bin/lib/Transforms/s3/jupyter.runfiles/__main__/lib/Transforms/SO3/SO3.py:22: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (self.isValid(R), "Rotation is invalid")
/home/jules/documents/code/ComputerVision/bazel-bin/lib/Transforms/s3/jupyter.runfiles/__main__/lib/Transforms/SO3/SO3.py:33: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (self.isValid(self.R), "Rotation is invalid")
/home/jules/documents/code/ComputerVision/bazel-bin/lib/Transforms/s3/jupyter.runfiles/__main__/lib/LinearAlgebra/matrix.py:6: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (len(matrix) > 2, f"Cannot transpose matrix with {len(matrix)} rows")
/home/jules/documents/code/ComputerVision/bazel-bin/lib/Transforms/s3/jupyter.runfiles/__main__/lib/LinearAlgebra/matrix.py:8: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (len(row) == len(matrix),

NameError: name 'rot_quat' is not defined

In [3]:
rot_quat.log_map()

[1.3993765903022615, 0.6996882951511307, 0.13993765903022615]

In [4]:
rot_mat.R

[[0.5873015873015873, 0.7936507936507936, 0.15873015873015872],
 [0.7936507936507936, -0.603174603174603, 0.07936507936507936],
 [0.15873015873015872, 0.07936507936507936, -0.984126984126984]]

In [10]:
import numpy as np
vec = np.array([1,2,3])
vec @ np.array(rot_mat.R)

array([-1.3015873 ,  3.50793651, -0.01587302])

## Linearizing the sphere
We may want to linearize the sphere around a base point $\mu$, such that the points on the sphere are represented in a local Euclidian tangent space $\tau_{\mu}$. The convenience comes from the linearity of tangent space, as mentioned in the Lie algebra section. $\tau_{\mu} gives a distorted "flattened" version of the unit sphere.

Since the maximal distance from $\mu$ to any point is $\pi$, we are generally only concerned with tangent vectors $v \in \tau_{\mu}$ where $||v|| < \le \pi$.

A point $x$ on the sphere can be mapped to $\tau_{\mu}$ via the centered log map
- $log_{\mu} (x) = (x - \mu (x^T \mu)) \frac{\theta}{sin(\theta)}$
- $\theta = arccos(x^T\mu)$

With the convention $\frac{0}{sin(0)} = 1$.

In [ ]:
from lib.Math.math_utils import dot, arccos, sin

def centered_log_map(x, mu):
    theta = arccos(dot(x, mu))
    safe_sin = theta / sin(theta) if theta != 0 else 1
    log_mu = (x - mu * dot(x, mu)) * safe_sin
    return log_mu

